In [1]:
import pandas as pd
import numpy as np
import preprocessor as p
import re
import string
import itertools
import requests

import nltk
#nltk.download()
from nltk.tokenize import word_tokenize
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

from tqdm.notebook import tqdm
tqdm.pandas()

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_colwidth', None)

In [2]:
raw = pd.read_csv('data.csv')
raw.shape

(108525, 6)

In [4]:
raw.head()

,tweet_id,created_at,author_id,username,text,reference_type
0,1620209968682254336,2023-01-30T23:58:53.000Z,185508421,rachmanfahadi,RT @NephiLaxmus: Saya\n\n- Pabrik motor listrik ini punya?\n+ Saya\n- Ojek online ini?\n+ Saya\n- Pembangkit listriknya?\n+ Saya\n- Pemasok batu ba…,['retweeted']
1,1620209586115604480,2023-01-30T23:57:22.000Z,458497906,66fathur,RT @NephiLaxmus: Saya\n\n- Pabrik motor listrik ini punya?\n+ Saya\n- Ojek online ini?\n+ Saya\n- Pembangkit listriknya?\n+ Saya\n- Pemasok batu ba…,['retweeted']
2,1620209295337078786,2023-01-30T23:56:12.000Z,954981728350695425,erusdhiana0118,"RT @giginpraginanto: Dia yang memproduksi, dia juga yang kasih subsidi. Dia pasti makin kaya raya dan berkuasa kalau program 3 periode berj…",['retweeted']
3,1620209006261460993,2023-01-30T23:55:03.000Z,1148010397951094784,AloelAsheen,"RT @giginpraginanto: Dia yang memproduksi, dia juga yang kasih subsidi. Dia pasti makin kaya raya dan berkuasa kalau program 3 periode berj…",['retweeted']
4,1620208586558435330,2023-01-30T23:53:23.000Z,1379546070783430661,DM_N4BYL,"RT @Nailazaara__: Erick Thohir melihat perkembangan renovasi ruang klinik dlm pemenuhan pelayanan atas pentingnya kesehatan pegawai.\n""Ke de…",['retweeted']


In [5]:
raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108525 entries, 0 to 108524
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   tweet_id        108525 non-null  int64 
 1   created_at      108525 non-null  object
 2   author_id       108525 non-null  int64 
 3   username        108525 non-null  object
 4   text            108525 non-null  object
 5   reference_type  83335 non-null   object
dtypes: int64(2), object(4)
memory usage: 5.0+ MB


# 1. Data Preprocessing
---

## 1.1 Filter Tweets

Drop retweeted Tweets to diminish duplication

In [6]:
raw['reference_type'].value_counts()

['retweeted']               66528
['replied_to']              14489
['quoted']                   2230
['quoted', 'replied_to']       88
Name: reference_type, dtype: int64

We will drop 66528 retweeted Tweets

In [7]:
df = raw[raw['reference_type'] != "['retweeted']"].reset_index(drop=True)

In [8]:
df.shape

(41997, 6)

## 1.2 Text Preprocessing

In [30]:
# Normalisasi Kata
url1 = (r'https://raw.githubusercontent.com/ksnugroho/klasifikasi-spam-sms/master/data/key_norm.csv')
url2 = (r'https://raw.githubusercontent.com/nasalsabila/kamus-alay/master/colloquial-indonesian-lexicon.csv')
url3 = (r'https://raw.githubusercontent.com/okkyibrohim/id-abusive-language-detection/master/kamusalay.csv')
url4 = (r'https://raw.githubusercontent.com/louisowen6/NLP_bahasa_resources/master/combined_slang_words.txt')
r = requests.get(url4)

kamus_normalisasi = pd.read_csv(url1)
kamus_alay1 = pd.read_csv(url2)
kamus_alay2 = pd.read_csv(url3, names=['slang','formal'])
kamus_slang = pd.json_normalize(r.json()).transpose().reset_index()
kamus_slang.columns = ['slang','formal']

def normalisasi(text):
  text = ' '.join([kamus_normalisasi[kamus_normalisasi['singkat'] == word]['hasil'].values[0] if (kamus_normalisasi['singkat'] == word).any() else word for word in text.split()])
  text = ' '.join([kamus_alay1[kamus_alay1['slang'] == word]['formal'].values[0] if (kamus_alay1['slang'] == word).any() else word for word in text.split()])
  text = ' '.join([kamus_alay2[kamus_alay2['slang'] == word]['formal'].values[0] if (kamus_alay2['slang'] == word).any() else word for word in text.split()])
  text = ' '.join([kamus_slang[kamus_slang['slang'] == word]['formal'].values[0] if (kamus_slang['slang'] == word).any() else word for word in text.split()])
  return text

def normalisasi_slang(text):
  text = ' '.join([kamus_alay1[kamus_alay1['slang'] == word]['formal'].values[0] if (kamus_alay1['slang'] == word).any() else word for word in text.split()])
  text = ' '.join([kamus_alay2[kamus_alay2['slang'] == word]['formal'].values[0] if (kamus_alay2['slang'] == word).any() else word for word in text.split()])
  text = ' '.join([kamus_slang[kamus_slang['slang'] == word]['formal'].values[0] if (kamus_slang['slang'] == word).any() else word for word in text.split()])
  return text

In [40]:
# Pipeline
def preprocessing(text):
  text = text.replace('\n', ' ').replace('\r', '') # replace new line into space
  text = text.replace('&amp;', ' ')
  p.set_options(p.OPT.URL, p.OPT.MENTION, p.OPT.HASHTAG, p.OPT.RESERVED)
  text = p.clean(text)
  text = normalisasi(text)
  #text = normalisasi_slang(text)
  return text

In [41]:
# Start preprocessing
df['text_cleaned'] = df['text'].progress_apply(preprocessing)

  0%|          | 0/41997 [00:00<?, ?it/s]

In [43]:
df[['text','text_cleaned']].sample(10)

,text,text_cleaned
36019,Semoga ada lagi perusahaan lain yang mulai masuk di daur ulang maupun repurposing baterai lithium dari kendaraan listrik. https://t.co/B4Hbt239L8,Semoga ada lagi perusahaan lain yang mulai masuk di daur ulang maupun repurposing baterai lithium dari kendaraan listrik.
14762,"@dawiguna @detikcom Jangan lupa mudahnya kepemilikan kendaraan (baik KR 2 maupun KR 4) dengan kredit jg menjadi pemicunya jika ditarik panjang ke belakang.\nBaru2 ini jg ditambah dengan munculnya kendaraan listrik dan ""dispesialkan"" dr ganjil genap serta subsidi kepemilikan KR listrik.","Jangan lupa mudahnya kepemilikan kendaraan (baik KR 2 maupun KR 4) dengan kredit juga menjadi pemicunya jika ditarik panjang ke belakang. Baru2 ini juga ditambah dengan munculnya kendaraan listrik dan ""dispesialkan"" dari ganjil genap serta subsidi kepemilikan KR listrik."
6457,"@alisyarief Belajar Ilmu kedokteran masih mahal. \n\nDaripada subsidi mobil listrik, lebih baik subsidi biaya kuliah kedokteran.","Belajar Ilmu kedokteran masih mahal. Daripada subsidi mobil listrik, lebih baik subsidi biaya kuliah kedokteran."
19756,"Selain ramah lingkungan juga motor listrik gaperlu ganti ganti oli, biaya perawatan yang jauh relatif lebih murah #PakaiMolis #PLNEmobilizeYou https://t.co/VVGHQkFq4j","Selain ramah lingkungan juga motor listrik tidak perlu ganti ganti oli, biaya perawatan yang jauh relatif lebih murah"
18182,Kebijakan subsidi kendaraan listrik dijadwalkan mulai berlaku per hari ini 20 Maret 2023. Motor dan mobil listrik apa saja yang dapat subsidi? https://t.co/UgSjtKgJin,Kebijakan subsidi kendaraan listrik dijadwalkan mulai berlaku per hari ini 20 Maret 2023. Motor dan mobil listrik apa saja yang dapat subsidi?
11966,@msaid_didu Dudu sdh ga sehat cara berpikir nya. Dgn kendaraan listrik otomatis pakaian bbm turun Dan subsidi jg turun. Dan subsidi pembelian hanya di berikan 1x saja saat beli. Subsidi bbm di berikan seumur hidup kendaraan selama masih beli bbm. Otak dipake du! Jgn benci aja di pelihara!,Dudu sudah tidak sehat cara berpikir nya. Dgn kendaraan listrik otomatis pakaian bbm turun Dan subsidi juga turun. Dan subsidi pembelian hanya di berikan 1x saja saat beli. Subsidi bbm di berikan seumur hidup kendaraan selama masih beli bbm. Otak dipakai du! Jgn benci saja di pelihara!
39919,"Hai Electrizen, fakta menarik dari kendaraan listrik ini ternyata mampu menghemat biaya operasional hingga 75 persen loh dibandingkan dengan kendaraan konvensional.\n\nTahukah kamu, beralih ke motor dan mobil listrik punya keuntungan berlipat dari sisi ekonomi. https://t.co/tAsIMoYIXB","Hai Electrizen, fakta menarik dari kendaraan listrik ini ternyata mampu menghemat biaya operasional hingga 75 persen loh dibandingkan dengan kendaraan konvensional. Tahukah kamu, beralih ke motor dan mobil listrik punya keuntungan berlipat dari sisi ekonomi."
196,"@KompasTV Subsidi motor listrik ini tidak tepat. Lebih berbau pertimbangan bisnis ketimbang upaya mengurangi polusi udara.\nApakah infrastruktur dan sarana pendukung utk kendaraan listrik sudah ada dan memadai? Ingat, harus ada sarana pengisian listrik batrei motor listrik yg tersebar luas","Subsidi motor listrik ini tidak tepat. Lebih berbau pertimbangan bisnis ketimbang upaya mengurangi polusi udara. Apakah infrastruktur dan sarana pendukung untuk kendaraan listrik sudah ada dan memadai? Ingat, harus ada sarana pengisian listrik batrei motor listrik yang tersebar luas"
5551,"Elektrifikasi kendaraan mampu menurunkan emisi namun perlu menyasar sektor transportasi publik, dan perlu memperhatikan dampak lingkungan dari pengerukkan sumber daya alam besar-besaran untuk menghasilkan bahan baku mobil listrik\nhttps://t.co/Pyxpvprfqf oleh @bumibungah","Elektrifikasi kendaraan mampu menurunkan emisi namun perlu menyasar sektor transportasi publik, dan perlu memperhatikan dampak lingkungan dari pengerukkan sumber daya alam besar-besaran untuk menghasilkan bahan baku mobil listrik oleh"
10791,Dengan pembangunan SPKLU ini 

In [44]:
# Save hasil preprocessing
df.to_csv('data/data_preprocessed.csv', index=False)